In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, recall_score
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.metrics import confusion_matrix 
from scipy import interp, stats
from imblearn.over_sampling import ADASYN 

### TO LOAD DATA

In [ ]:
# TO IMPORT CVS FILES (REGARDING FREQUENCY OF FEATURES)
all_features_list_df_cub = pd.read_csv("training_cubic_all_features_list_result.csv",index_col=False)
all_features_count_df_cub = all_features_list_df_cub.stack().value_counts() # it returns a df with the frequency for each features

In [ ]:
# TO IMPORT CVS FILE AND CREATE A PD DATAFRAME WITH ONLY THE FIRST n SELECTED FEATURES - TRAINING DATASET

first_n_features_to_select_cub = 7 # choose the value

# load the original dataset
training_dataframe_df_cub = pd.read_csv("training - cubic after WEKA CfsSubsetEval.csv",index_col='exam')
size_mapping = {"codeletion":0,"noncodeletion":1} # MAPPING for outcome 
training_dataframe_df_cub["outcome"] = training_dataframe_df_cub["outcome"].map(size_mapping)

training_feature_names_cub = [x[2:-2] for x in [*all_features_count_df_cub.index]]
training_selected_features_cub = training_feature_names_cub[:first_n_features_to_select_cub]
training_New_dataframe_cub = training_dataframe_df_cub[training_selected_features_cub]
training_New_dataframe_cub["outcome"] = training_dataframe_df_cub["outcome"] 
training_dataframe_with_selected_features_df_cub = training_New_dataframe_cub

In [ ]:
# TO IMPORT CVS FILE AND CREATE A PD DATAFRAME WITH ONLY THE FIRST n SELECTED FEATURES - TESTING DATASET

first_n_features_to_select_cub = 7 # choose the value

# load the original dataset
testing_dataframe_df_cub = pd.read_csv("testing - cubic.csv",index_col='exam', encoding = "ISO-8859-1") # insert the all original dataset
size_mapping = {"codeletion":0,"noncodeletion":1} # MAPPING for outcome 
testing_dataframe_df_cub["outcome"] = testing_dataframe_df_cub["outcome"].map(size_mapping)

testing_feature_names_cub = [x[3:-3] for x in [*all_features_count_df_cub.index]]
testing_selected_features_cub = testing_feature_names_cub[:first_n_features_to_select_cub]
testing_New_dataframe_cub = testing_dataframe_df_cub[testing_selected_features_cub]
testing_New_dataframe_cub["outcome"] = testing_dataframe_df_cub["outcome"]
testing_dataframe_with_selected_features_df_cub = testing_New_dataframe_cub

In [ ]:
print ("The chosen features are:", [x[1:-1] for x in [*training_selected_features_cub]])

## Training the model on the training dataset and testing the model on validation dataset

In [ ]:
model_cub = RandomForestClassifier(random_state=1, n_estimators=100) # Choose the model

In [ ]:
# To rename dataframes into X_train_cub, Y_train_cub, X_test_cub, Y_test_cub (numpy arrays)

Y_train_cub = training_dataframe_with_selected_features_df_cub['outcome']
X_train_cub = training_dataframe_with_selected_features_df_cub.drop('outcome',axis=1)

Y_test_cub = testing_dataframe_with_selected_features_df_cub['outcome']
X_test_cub = testing_dataframe_with_selected_features_df_cub.drop('outcome',axis=1)

In [ ]:
#StandardScaler
ss = StandardScaler() 
X_train_SS_np_cub = ss.fit_transform(X_train_cub) 
X_train_SS_cub = pd.DataFrame(X_train_SS_np_cub, index=X_train_cub.index, columns=X_train_cub.columns)
X_test_SS_np_cub = ss.transform(X_test_cub) 
X_test_SS_cub = pd.DataFrame(X_test_SS_np_cub, index=X_test_cub.index, columns=X_test_cub.columns)

# ADASYN
sm = ADASYN(random_state=1)
X_train_SS_balanced_np_cub, Y_train_balanced_np_cub = sm.fit_sample(X_train_SS_cub, Y_train_cub)
X_train_SS_balanced_cub = pd.DataFrame(X_train_SS_balanced_np_cub, columns=X_train_SS_cub.columns)
Y_train_balanced_cub = pd.DataFrame(Y_train_balanced_np_cub, columns=["outcome"])

# Fitting the model
model_cub.fit (X_train_SS_balanced_cub, Y_train_balanced_cub)

# Compute predictions, probabilities and accuracy
predictions_cub = model_cub.predict(X_test_SS_cub)
probabilities_cub = model_cub.predict_proba(X_test_SS_cub)
accuracy_cub = accuracy_score(Y_test_cub, predictions_cub)

# Compute AUC
fpr_cub, tpr_cub, threshold_cub = roc_curve(Y_test_cub, np.array(probabilities_cub)[:,1])
roc_auc_cub = auc(fpr_cub, tpr_cub)

In [ ]:
# Rename the values for bootstrap code and De-Long test
y_true_cub = np.array(Y_test_cub)
y_pred_cub = np.array(predictions_cub)
y_prob_cub = np.array(probabilities_cub)[:,1]

In [ ]:
# print Confusion Matrix
print ("Confusion matrix for cubic features: \n", confusion_matrix(y_true_cub, y_pred_cub))

In [ ]:
# Perform BOOTSTRAP with y_true, predictions, probabilities

n_bootstraps = 10000
rng_seed = 1  # control reproducibility

bootstrapped_acc_cub = []
bootstrapped_auc_cub = []
bootstrapped_sens_cub = []
bootstrapped_spec_cub = []

bootstrapped_tpr_cub = []
bootstrapped_fpr_cub = []
bootstrapped_thr_cub = []

bootstrapped_tprs_cub = []

mean_fpr = np.linspace(0, 1, 100)

rng = np.random.RandomState(rng_seed)

for i in range(n_bootstraps):
    # bootstrap by sampling with replacement on the prediction indices
    indices_0=np.where(y_true_cub == 0)
    indices_1=np.where(y_true_cub == 1)
    
    # 'balanced bootstrapping'
    random_indices_0=rng.choice(indices_0[0],len(indices_0[0]))
    random_indices_1=rng.choice(indices_1[0],len(indices_0[0]))
    random_indices=np.concatenate((random_indices_0,random_indices_1), axis=None)

    acc_cub = accuracy_score(y_true_cub[random_indices], y_pred_cub[random_indices])
    auc_cub = roc_auc_score(y_true_cub[random_indices], y_prob_cub[random_indices])
    sens_cub = recall_score(y_true_cub[random_indices], y_pred_cub[random_indices], pos_label=1)
    spec_cub = recall_score(y_true_cub[random_indices], y_pred_cub[random_indices], pos_label=0)
    
    fpr_cub, tpr_cub, threshold_cub = roc_curve(y_true_cub[random_indices], y_prob_cub[random_indices])
    
    interp_tpr_cub = interp(mean_fpr, fpr_cub, tpr_cub)
    interp_tpr_cub[0] = 0.0
    bootstrapped_tprs_cub.append(interp_tpr_cub)
    
    bootstrapped_acc_cub.append(acc_cub)
    bootstrapped_auc_cub.append(auc_cub)
    bootstrapped_sens_cub.append(sens_cub)
    bootstrapped_spec_cub.append(spec_cub)

metrics distributions for bootstrapping steps

In [ ]:
plt.figure(figsize=(10, 15))

plt.subplot(2,2,1)
plt.hist(bootstrapped_acc_cub)
plt.title('Acc cub')

plt.subplot(2,2,2)
plt.hist(bootstrapped_auc_cub)
plt.title('AUC cub')

plt.subplot(2,2,3)
plt.hist(bootstrapped_sens_cub)
plt.title('Sens cub')

plt.subplot(2,2,4)
plt.hist(bootstrapped_spec_cub)
plt.title('Spec cub')

plt.show()

distr normality test (Shapiro-Wilcoxon)

In [ ]:
print ('Acc cub: ', stats.shapiro(bootstrapped_acc_cub))
print ('AUC cub: ', stats.shapiro(bootstrapped_auc_cub))
print ('Sens cub: ', stats.shapiro(bootstrapped_sens_cub))
print ('Spec cub: ', stats.shapiro(bootstrapped_spec_cub))

p-values are small -> distr is not normal -> estimation should be represented as median (low_percentile, up_percentile)

In [ ]:
print ('Acc cub: {} ({}, {})'.format(np.median(bootstrapped_acc_cub), np.percentile(bootstrapped_acc_cub, 2.5), np.percentile(bootstrapped_acc_cub, 97.5)))
print ('AUC cub: {} ({}, {})'.format(np.median(bootstrapped_auc_cub), np.percentile(bootstrapped_auc_cub, 2.5), np.percentile(bootstrapped_auc_cub, 97.5)))
print ('Sens cub: {} ({}, {})'.format(np.median(bootstrapped_sens_cub), np.percentile(bootstrapped_sens_cub, 2.5), np.percentile(bootstrapped_sens_cub, 97.5)))
print ('Spec cub: {} ({}, {})'.format(np.median(bootstrapped_spec_cub), np.percentile(bootstrapped_spec_cub, 2.5), np.percentile(bootstrapped_spec_cub, 97.5)))

## ROC CURVE AND AUC

In [ ]:
# ROC CURVE

fig, ax = plt.subplots(figsize=(10,10))  

plt.title('ROC Validation dataset')

plt.plot([0, 1], [0, 1], linestyle='--', lw=2, color='r', alpha=.8)

mean_tpr_cub = np.median(bootstrapped_tprs_cub, axis=0)  
mean_tpr_cub[-1] = 1.0                
plt.plot(mean_fpr, mean_tpr_cub, color='b', 
        label=r'Median ROC (AUC = %0.2f)' % (np.median(bootstrapped_auc_cub)), 
        lw=2, alpha=.8)

tprs_upper = np.percentile(bootstrapped_tprs_cub, 2.5,  axis = 0)
tprs_lower = np.percentile(bootstrapped_tprs_cub, 97.5, axis = 0)
plt.fill_between(mean_fpr, tprs_lower, tprs_upper, color='grey', alpha=.2, label='95 % CI')

plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')

plt.xlim([0, 1])
plt.ylim([0, 1])

plt.legend(loc="lower right")
plt.show()